<a href="https://colab.research.google.com/github/smiliyo55/psoCNN_Brain_Tumor/blob/main/psoCNN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install import_ipynb
!pip install torchsummary
import import_ipynb

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary

import numpy as np

from copy import deepcopy

import utils1
import particle1
from population1 import Population

In [ ]:
from torchvision.datasets import ImageFolder

In [ ]:
class psoCNN:
  def __init__(self, dataset, n_iter, pop_size, batch_size, epochs, min_layer, max_layer, \
      conv_prob, pool_prob, fc_prob, max_conv_kernel, max_out_ch, max_fc_neurons, dropout_rate):

    self.pop_size = pop_size
    self.n_iter = n_iter
    self.epochs = epochs

    self.batch_size = batch_size
    self.gBest_acc = np.zeros(n_iter)
    self.gBest_test_acc = np.zeros(n_iter)

    dataset = "Brain Tumor MRI Dataset"
    input_width = 224
    input_height = 224
    input_channels = 3
    output_dim = 4

    # Define transformations
    transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

    # Load MNIST dataset
    trainset = ImageFolder('/content/drive/MyDrive/Colab Notebooks/psoCNN2/Training', transform=transform)

    testset = ImageFolder('/content/drive/MyDrive/Colab Notebooks/psoCNN2/Testing', transform=transform)



    # Create data loaders
    self.trainloader = torch.utils.data.DataLoader(trainset, batch_size=self.batch_size,
                                              shuffle=True)
    self.testloader = torch.utils.data.DataLoader(testset, batch_size=self.batch_size,
                                            shuffle=False)

    print("Initializing population...")
    self.population = Population(pop_size, min_layer, max_layer, input_width, input_height, input_channels,
                                  conv_prob, pool_prob, fc_prob, max_conv_kernel, max_out_ch, max_fc_neurons, output_dim)

    print("Verifying accuracy of the current gBest...")
    print(self.population.particle[0])
    self.gBest = deepcopy(self.population.particle[0])
    self.gBest.model_compile(dropout_rate)
    hist = self.gBest.model_fit(self.trainloader, batch_size=batch_size, epochs=epochs)
    test_metrics = self.gBest.evaluate(self.testloader, batch_size)
    self.gBest.model_delete()

    self.gBest_acc[0] = hist['accuracy'][-1]
    self.gBest_test_acc[0] = test_metrics['accuracy']

    self.population.particle[0].acc = hist['accuracy'][-1]
    self.population.particle[0].pBest.acc = hist['accuracy'][-1]

    print("Current gBest acc: " + str(self.gBest_acc[0]) + "\n")
    print("Current gBest test acc: " + str(self.gBest_test_acc[0]) + "\n")

    print("Looking for a new gBest in the population...")
    for i in range(1, self.pop_size):
        print('Initialization - Particle: ' + str(i+1))
        print(self.population.particle[i])

        self.population.particle[i].model_compile(dropout_rate)
        hist = self.population.particle[i].model_fit(self.trainloader, batch_size=batch_size, epochs=epochs)
        #self.population.particle[i].model_delete()

        self.population.particle[i].acc = hist['accuracy'][-1]
        self.population.particle[i].pBest.acc = hist['accuracy'][-1]

        if self.population.particle[i].pBest.acc >= self.gBest_acc[0]:
            print("Found a new gBest.")
            self.gBest = deepcopy(self.population.particle[i])
            self.gBest_acc[0] = self.population.particle[i].pBest.acc
            print("New gBest acc: " + str(self.gBest_acc[0]))

            #self.gBest.model_compile(dropout_rate)
            test_metrics = self.gBest.evaluate(self.testloader, batch_size=batch_size)
            self.gBest_test_acc[0] = test_metrics['accuracy']
            print("New gBest test acc: " + str(self.gBest_acc[0]))
            self.gBest.model_delete()

        self.population.particle[i].model_delete()

  def fit(self, Cg, dropout_rate):
    for i in range(1, self.n_iter):
      gBest_acc = self.gBest_acc[i-1]
      gBest_test_acc = self.gBest_test_acc[i-1]

      for j in range(self.pop_size):
        print('Iteration: ' + str(i) + ' - Particle: ' + str(j+1))

        # Update particle velocity
        self.population.particle[j].velocity(self.gBest.layers, Cg)

        # Update particle architecture
        self.population.particle[j].update()

        print('Particle NEW architecture: ')
        print(self.population.particle[j])

        # Compute the acc in the updated particle
        self.population.particle[j].model_compile(dropout_rate)
        hist = self.population.particle[j].model_fit(self.trainloader, batch_size=self.batch_size, epochs=self.epochs)
        #self.population.particle[j].model_delete()

        self.population.particle[j].acc = hist['accuracy'][-1]

        f_test = self.population.particle[j].acc
        pBest_acc = self.population.particle[j].pBest.acc

        if f_test >= pBest_acc:
            print("Found a new pBest.")
            print("Current acc: " + str(f_test))
            print("Past pBest acc: " + str(pBest_acc))
            pBest_acc = f_test
            self.population.particle[j].pBest = deepcopy(self.population.particle[j])
            self.population.particle[j].pBest.model_delete()

            if pBest_acc >= gBest_acc:
                print("Found a new gBest.")
                gBest_acc = pBest_acc
                self.gBest = deepcopy(self.population.particle[j])

                #self.gBest.model_compile(dropout_rate)
                #hist = self.gBest.model_fit(self.trainloader, batch_size=self.batch_size, epochs=self.epochs)
                test_metrics = self.gBest.evaluate(self.testloader, batch_size=self.batch_size)
                self.gBest.model_delete()
                gBest_test_acc = test_metrics['accuracy']

        self.population.particle[j].model_delete()

      self.gBest_acc[i] = gBest_acc
      self.gBest_test_acc[i] = gBest_test_acc

      print("Current gBest acc: " + str(self.gBest_acc[i]))
      print("Current gBest test acc: " + str(self.gBest_test_acc[i]))

  def fit_gBest(self, batch_size, epochs, dropout_rate):
    print("\nFurther training gBest model...")
    self.gBest.model_compile(dropout_rate)
    print(self.gBest)
    self.gBest.model_summary()

    trainable_count = particle1.count_parameters(self.gBest.model)
    print("gBest's number of trainable parameters: " + str(trainable_count))

    metrics = self.gBest.model_fit_complete(self.trainloader, self.testloader, batch_size=batch_size, epochs=epochs)

    print("\ngBest model loss in the test set: " + str(metrics['val loss'][-1]) + " - Test set accuracy: " + str(metrics['val accuracy'][-1]))

    self.gBest.model_delete()

    return trainable_count, metrics